# Chat Completion API

In [ ]:
!pip install openai

# 환경설정
1.openai 사이트에서 API KEY발급
google colab -secret탭에 `OPENAI_API_KEY`

In [1]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')



In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 유연석이야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [4]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 유연석님! 만나서 반갑습니다. 어떤 도움이 필요하신가요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고,표현 조정

- 프랑스 AFP 속보시스템에서 도입되어 사용


In [5]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before='졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'
# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목:'전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다.'
- 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 교정해주세요.

제목: {title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: 졸라 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환
- 교정 제목: 힘든 작업으로 끼니를 거르는 노동자들의 고충


In [ ]:
# 함수 리펙토링
def correct_news_title(title_before,model='gpt-4o-mini',temperature=1,top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목:'전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다.'
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content
title_before='주말 미친 폭우 예상, 모두들 무사하시길..'
output=correct_news_title(title_before,)
print(output)

- 원래 제목: 주말 미친 폭우 예상, 모두들 무사하시길..
- 교정 제목: "주말 폭우 예상, 안전 유의하시기 바랍니다"


## 영문텍스트에 anki 단어장 만들기

In [ ]:
# 함수 리펙토링
def correct_news_title(title_before,model='gpt-4o-mini',temperature=1,top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목:'전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다.'
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content
title_before='주말 미친 폭우 예상, 모두들 무사하시길..'
output=correct_news_title(title_before,)
print(output)

## 영문텍스트에 anki단어장 만들기

In [ ]:
# 함수 리펙토링
def dating_coach(prompt,model='gpt-4o-mini',temperature=1,top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
   LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
   너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
   각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
   각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.
    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


    ### Output Format ###
    각 카드: 단어 또는 어구;뜻;품사;영어 예문
    모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
    불필요한 부가설명, 줄바꿈, 문장 없음

    ### Examples ###
    - The committee will convene next week to discuss the new policy proposals and reach a consensus.
    - convene;to come together for a meeting;verb;The committee will convene next week.
      consensus;general agreement;noun;The group reached a consensus after a long discussion.
      proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

    - 'The scientist will analyze the data before publishing the results.'
    - 'analyze;to examine something carefully in order to understand it;verb;The scientist will analyze the data before publishing the results.
    data;facts or information used for analysis;noun;The data showed a clear trend in temperature changes.
    result;the outcome of an action or experiment;noun;The results of the experiment were surprising.'

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content
title_before='주말 미친 폭우 예상, 모두들 무사하시길..'
output=correct_news_title(title_before,)
print(output)

## 연애코치 ReAct

In [ ]:
def dating_coach(prompt,model='gpt-4o-mini',temperature=1,top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
   <<System Instruction>>
   어떤상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해주세요.

   <<Output Format>>
   1.상황분석:

   2.행동계획:

   3.실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content

In [ ]:
print(dating_coach('제가 어제 친구들이 놀다가 여자친구랑 연락을 잘 못해서 단단히 삐져있는 상황입니다. 이 위기를 어떻게 극복해야 할까요?'))

1. 상황 분석:
   - 여자친구와 연락이 원활하지 않아 서로의 감정을 다치게 한 상황입니다. 친구들과의 시간이 여자친구에게 소홀함으로 비쳤을 가능성이 크며, 이로 인해 그녀가 상처받았거나 불안함을 느꼈을 수 있습니다. 감정이 격해질 대로 격해져 있는 상황이므로 신중한 접근이 필요합니다.

2. 행동 계획:
   - **소통:** 먼저 그녀에게 연락하여 상황을 진심으로 사과해주세요. 연락이 원활하지 않았던 점을 사과하고, 그녀의 감정을 이해하고 존중한다는 메시지를 전하세요.
   - **경청:** 그녀가 어떻게 느끼고 있는지 들어보며 감정에 공감해주는 것이 중요합니다. 그녀의 말에 귀 기울여 주고, 자신의 입장을 변명하기보다 그녀의 감정을 먼저 우선시하세요.
   - **보상할 방법 제안:** 그녀의 기분을 좋게 할 수 있는 작은 계획을 제안해보세요. 예를 들어, 다음 데이트에서는 그녀가 좋아하는 장소에 가거나, 특별한 이벤트를 준비하는 등의 방법이 있습니다.

3. 실행:
   - 먼저 그녀에게 문자나 전화를 통해 사과의 메시지를 전하세요. 예를 들어, "어제 연락이 잘 안 돼서 정말 미안해. 네가 어떻게 느꼈는지 듣고 싶어."라는 식의 내용으로 시작하세요.
   - 대화가 이어지면 그녀의 이야기를 잘 들어주고, 어떤 구체적인 행동이 그녀에게 기쁨을 줄 수 있을지 생각해보세요. 예를 들어, 데이트 장소를 정해두고 "이번 주말에 너와 함께 하고 싶은 활동이 있어. 널 위해 준비했어."라는 말을 덧붙이면 좋습니다. 
   - 마지막으로, 그녀에게 진정한 관심과 애정을 보여주는 것으로 상황을 극복할 수 있습니다. 피해를 줄이기 위해 본인의 행동을 돌아보고, 앞으로 소통을 더 원활히 하겠다는 약속도 함께 해보세요. 

이 과정을 통해 문제를 해결하고, 관계의 신뢰를 더욱 깊게 할 수 있을 것입니다.


# 냉털 마스터 ReAct

- 사용자는 냉장고에 남아있는 음식 재료를 알려주면 ,LLM 은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action 기법을 끌어낼수있는 적절한 프롬프팅을 작성하세요.

In [ ]:
def refrigerator(prompt,model='gpt-4o-mini',temperature=1,top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
   <<System Instruction>>
   1.ReAct기법을 적용해서 재료를 받으면 그재료에 맞는 음식을 제시해줘
   2. 앞에서제시한 음식의 양도 표시해주고 step1,step2..이런형식으로 요리방법을 알려줘
   3. 요리초보자도 알수있게 쉽고 자세히 알려줘

   <<Output Format>>
    재료:
    떡볶이 떡: 300g (밀떡 또는 쌀떡)
    어묵: 2~3장 (한입 크기로 썰기)
    양파: 1/2개 (굵게 썰기)
    대파: 1/2대 (어슷하게 썰기)
    물: 2컵(400ml)
    삶은 달걀 (선택): 1~2개
    **양념장**
    고추장: 2큰술
    고춧가루: 1큰술
    설탕: 1~1.5큰술 (혹은 올리고당)
    간장: 1큰술
    다진 마늘: 1큰술
    후추: 약간

    ### Examples ###
    **Input**
    떡볶이 떡,어묵,양파,대파,삶은 달걀
    ** Output**
    재료:
    떡볶이 떡: 300g (밀떡 또는 쌀떡)
    어묵: 2~3장 (한입 크기로 썰기)
    양파: 1/2개 (굵게 썰기)
    대파: 1/2대 (어슷하게 썰기)
    물: 2컵(400ml)
    삶은 달걀 (선택): 1~2개
    **양념장**
    고추장: 2큰술
    고춧가루: 1큰술
    설탕: 1~1.5큰술 (혹은 올리고당)
    간장: 1큰술
    다진 마늘: 1큰술
    후추: 약간

    step1떡 준비
    떡이 딱딱하면 미리 찬물에 10~20분 정도 담가 두세요.

    step2육수 내기 (선택)
    멸치, 다시마 등으로 간단하게 육수를 만들면 더 감칠맛나요. 그냥 물을 사용해도 무방합니다.

    step3 양념장 만들기
    고추장, 고춧가루, 설탕, 간장, 다진 마늘을 볼에 넣고 잘 섞어주세요.

    step4 끓이기
    냄비에 물과 양념장을 넣고 풀어준 후 끓입니다.
    끓기 시작하면 떡, 어묵, 양파를 넣고 중불에서 끓여 주세요.

    step5 졸이기
    떡이 말랑해질 때까지 저어가며 끓입니다. 물이 너무 졸아들면 조금씩 더 추가해 주세요.
    대파와 삶은 달걀을 넣고 한 번 더 끓여줍니다.

    step6 마무리
    떡이 완전히 말랑하고, 양념이 적당히 졸았으면 불을 끄고 그릇에 담아내면 완성!

    """


    user_message = f"""
    냉장고 안에 있는 재료는
    {prompt}다음과 같습니다.
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )
    return response.choices[0].message.content



In [ ]:
from IPython.display import Markdown
display(Markdown(refrigerator('마늘 빵 케첩 양파 파')))

재료:
- 마늘빵: 4조각
- 케첩: 1/2컵
- 양파: 1/2개 (잘게 썰기)
- 파: 1/2대 (어슷하게 썰기)

### 요리법: 간단한 마늘빵 케첩 토스트

step1 마늘빵 준비하기  
마늘빵이 두께가 두꺼우면 포크로 눌러 약간 눌려 주세요. 오븐 또는 토스트기를 사용할 준비를 합니다.

step2 양파 준비하기  
양파를 잘게 썰어 주세요. 큰 덩어리로 남지 않도록 주의하세요. 

step3 양파 볶기  
팬에 기름을 두르고, 중불에서 썰어놓은 양파를 넣고 볶아 주세요. 양파가 반투명해질 때까지 볶으면 됩니다. (약 3-5분 정도)

step4 마늘빵 올리기  
마늘빵을 오븐 팬이나 토스트기에 올리고, 볶은 양파를 마늘빵의 위에 evenly spread 해줍니다.

step5 케첩 뿌리기  
볶은 양파 위에 케첩을 적당히 뿌려 주세요. 취향에 따라 더 많이 뿌려도 괜찮습니다.

step6 굽기  
오븐을 180도로 예열한 후, 준비한 마늘빵을 오븐에 넣고 5-10분 정도 구워줍니다. 마늘빵의 가장자리가 바삭해지면 좋습니다.

step7 마무리  
오븐에서 꺼내고, 썰어둔 파를 위에 뿌린 후 이제 드디어 완성! 따뜻하게 먹으면 최고의 맛을 느낄 수 있습니다. 

맛있게 드세요!

## 강사님 프롬프

In [ ]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물로 별도록 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [ ]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', '표고버섯', '명란젓', '된장']
display(Markdown(fridge_raid_master(user_foods)))

## 면접질문 생성 JSON 출력

In [ ]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI 서비스개발의 전문가로써 해당분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.


    <<지시사항>>
    - 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요
    - 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
    - 각 스킬별로 질문/답변3개씩 만들어주세요

   <<출력형식>>
   출력은 json형식으로만 반환되어야 합니다.
   {{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

    """

    user_message = f"""
채용공고:
{job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [ ]:
job_posting="""
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>   (MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |

"""
output=job_interview(job_posting)
print(type(output))
print(job_interview(job_posting))

<class 'str'>
{
    "hard_skill": [
        {
            "question": "AI 모델의 배포를 위한 인프라 구축 시 어떤 요소를 가장 중요하게 고려합니까?",
            "answer": "AI 모델의 배포를 위한 인프라 구축 시, 확장성, 가용성, 보안성을 가장 중요하게 고려해야 합니다. 또한, 다양한 환경에서의 테스트 및 운영이 용이하도록 CI/CD 파이프라인을 통합하는 것이 중요합니다."
        },
        {
            "question": "Docker와 Kubernetes를 사용한 경험에 대해 설명해 주세요.",
            "answer": "저는 Docker를 사용하여 애플리케이션을 컨테이너화했으며, Kubernetes를 통해 클러스터를 관리하고 오케스트레이션을 수행했습니다. 이를 통해 확장성과 자원 효율성을 높였고, DevOps 환경에서 제너릭 지속적 통합 및 배포 프로세스를 구축했습니다."
        },
        {
            "question": "MLflow 또는 Kubeflow를 사용하여 모델 서빙 환경을 구축한 경험이 있습니까?",
            "answer": "네, MLflow를 사용하여 모델의 버전 관리 및 추적을 실시했고, Kubeflow를 통해 머신러닝 파이프라인을 자동화하여 실험과 배포를 원활하게 진행했습니다. 이를 통해 모델 성능을 지속적으로 모니터링할 수 있도록 시스템을 설계했습니다."
        }
    ],
    "soft_skill": [
        {
            "question": "AI 모델 배포와 관련하여 알고리즘 개발자와 협업할 때 가장 중요한점은 무엇이라고 생각하나요?",
            "answer": "모델 배포 과정에서 알고리즘 개발자와의 효과적인 커뮤니케이션이 가장 중요합니다. 기술적인 요구사항 및 모델 성능에 대해 명확하게 전달하고, 서로의 의

In [ ]:
import json

#응답데이터가 json형식아니라면 오류발생!
data=json.loads(output)
print(type(data))
hard_skill_qa=data['hard_skill']
soft_skill_qa=data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'AI 모델의 배포를 위한 인프라 구축 프로세스를 설명해주세요.', 'answer': 'AI 모델의 배포를 위한 인프라는 기본적인 설계, 기술 선정, 환경 구축, 및 테스트의 단계를 포함합니다. 먼저, 모델의 요구 사항을 분석하고 적절한 VM 또는 클라우드 인프라를 결정한 후, 컨테이너 기술인 Docker를 사용하여 모델을 패키징합니다. Kubernetes를 활용하여 모델 배포에 대한 오케스트레이션을 수행하며, Jenkins나 ArgoCD를 사용하여 CI/CD 프로세스를 구축합니다. 마무리로, 배포 후 모니터링과 로깅 시스템을 설정하여 모델의 성능을 지속적으로 추적합니다.'}, {'question': '머신러닝 시스템의 학습, 배포, 테스트 환경을 어떻게 자동화할 수 있나요?', 'answer': '자동화하기 위해 CI/CD 도구를 활용하여 파이프라인을 구성합니다. 예를 들어, Jenkins는 코드 변경 시 자동으로 모델을 학습하고, 테스트하는 프로세스를 설정할 수 있습니다. 배포 단계에서는 Kubernetes의 Helm 차트를 사용하여 모델을 손쉽게 배포하고, 이를 통해 실시간으로 모델 변경을 적용할 수 있습니다. 최종적으로, 운영 환경에서는 모니터링 도구를 사용하여 모델의 성능과 상태를 확인하고, 문제 발생 시 자동으로 롤백할 수 있도록 구성합니다.'}, {'question': '오픈소스 기반 플랫폼과 내재화 구축 경험에 대해 설명해 주세요.', 'answer': 'MLflow와 Kubeflow를 이용한 프로젝트 경험이 있습니다. MLflow로 실험 추적 및 모델 서빙을 처리하며, Kubeflow를 활용해 포털을 구축해 조율할 수 있었습니다. 이 과정에서 각 툴의 특성을 이해하고, 기존 인프라와 통합하는 과정에서 발생한 문제를 해결하기 위해 기술 문서를 분석하고 커뮤니티에 질문을 올려 도움을 받았습니다. 최종적으로, 사용자 친화적인 환경을 제공하여 팀원들의 AI 모델 개발 속도를 크게 향상시킬 수 있었습니

In [ ]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
"""

    user_message = f"""

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

<< 채용공고 >>
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [ ]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))
print(output)

In [ ]:
import json

# 응답데이터가 json형식이 아니라면 오류발생!
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)